# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san ignacio,17.1561,-89.0714,307.23,52,20,2.06,BZ,2024-09-01
1,1,avarua,-21.2078,-159.7750,294.18,94,75,7.72,CK,2024-09-01
2,2,edinburgh of the seven seas,-37.0676,-12.3116,283.03,67,100,10.06,SH,2024-09-01
3,3,khasab,26.1799,56.2477,304.03,59,75,2.31,OM,2024-09-01
4,4,huaral,-11.4950,-77.2078,293.10,65,49,3.44,PE,2024-09-01


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_all = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    marker = 'd',
    s = city_data_df['Humidity'],
    geo = True,
    tiles = "OSM"
)

# Display the map
map_all

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[city_data_df['Humidity'] < 50][city_data_df['Max Temp'] < 293][city_data_df['Lat'] < 0]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

C:\Users\audre\AppData\Local\Temp\ipykernel_3364\352992597.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_df = city_data_df[city_data_df['Humidity'] < 50][city_data_df['Max Temp'] < 293][city_data_df['Lat'] < 0]
C:\Users\audre\AppData\Local\Temp\ipykernel_3364\352992597.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_df = city_data_df[city_data_df['Humidity'] < 50][city_data_df['Max Temp'] < 293][city_data_df['Lat'] < 0]


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,marondera,-18.1853,31.5519,282.25,46,0,4.39,ZW,2024-09-01
67,67,juan de ayolas,-27.4000,-56.9000,291.78,49,89,3.57,PY,2024-09-01
191,191,brinkmann,-30.8659,-62.0374,291.52,37,0,1.30,AR,2024-09-01
269,269,villa canas,-34.0057,-61.6076,289.23,10,0,3.11,AR,2024-09-01
286,286,chiredzi,-21.0500,31.6667,288.27,38,0,1.46,ZW,2024-09-01
336,336,carnarvon,-24.8667,113.6333,291.13,42,0,6.13,AU,2024-09-01
385,385,bahia blanca,-38.7196,-62.2724,289.88,37,100,6.56,AR,2024-09-01
400,400,ritchie,-29.0380,24.6028,289.55,42,0,2.11,ZA,2024-09-01
404,404,gokwe,-18.2048,28.9349,287.13,41,0,5.54,ZW,2024-09-01
412,412,petauke,-14.2426,31.3253,290.92,47,1,4.19,ZM,2024-09-01


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

C:\Users\audre\AppData\Local\Temp\ipykernel_3364\3389241433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
39,marondera,ZW,-18.1853,31.5519,46,
67,juan de ayolas,PY,-27.4000,-56.9000,49,
191,brinkmann,AR,-30.8659,-62.0374,37,
269,villa canas,AR,-34.0057,-61.6076,10,
286,chiredzi,ZW,-21.0500,31.6667,38,
336,carnarvon,AU,-24.8667,113.6333,42,
385,bahia blanca,AR,-38.7196,-62.2724,37,
400,ritchie,ZA,-29.0380,24.6028,42,
404,gokwe,ZW,-18.2048,28.9349,41,
412,petauke,ZM,-14.2426,31.3253,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'apiKey':geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
marondera - nearest hotel: No hotel found
juan de ayolas - nearest hotel: Hotel Nacional de Turismo Ayolas
brinkmann - nearest hotel: No hotel found
villa canas - nearest hotel: Colón
chiredzi - nearest hotel: Nesbitt Arms Hotel
carnarvon - nearest hotel: No hotel found
bahia blanca - nearest hotel: Hotel Austral Bahía
ritchie - nearest hotel: No hotel found
gokwe - nearest hotel: No hotel found
petauke - nearest hotel: Chimwemwe Lodge
hacienda huancane - nearest hotel: No hotel found
ingeniero jacobacci - nearest hotel: Gran Hotel Argentino
luena - nearest hotel: No hotel found
kambove - nearest hotel: No hotel found
glencoe - nearest hotel: Royal Country Inn
diamantina - nearest hotel: Pousada do Garimpo
tshabong - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
39,marondera,ZW,-18.1853,31.5519,46,No hotel found
67,juan de ayolas,PY,-27.4000,-56.9000,49,Hotel Nacional de Turismo Ayolas
191,brinkmann,AR,-30.8659,-62.0374,37,No hotel found
269,villa canas,AR,-34.0057,-61.6076,10,Colón
286,chiredzi,ZW,-21.0500,31.6667,38,Nesbitt Arms Hotel
336,carnarvon,AU,-24.8667,113.6333,42,No hotel found
385,bahia blanca,AR,-38.7196,-62.2724,37,Hotel Austral Bahía
400,ritchie,ZA,-29.0380,24.6028,42,No hotel found
404,gokwe,ZW,-18.2048,28.9349,41,No hotel found
412,petauke,ZM,-14.2426,31.3253,47,Chimwemwe Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
map_select = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    marker = 'o',
    s = city_data_df['Humidity'],
    geo = True,
    tiles = "OSM",
    hover_cols = ['City','Country','Lat','Lng','Humidity','Hotel Name']
)

# Display the map
map_select

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_size,City,Country,Humidity,Hotel Name)